<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [1]:
#|output: asis
#| echo: false
show_doc(History)

---

[source](https://github.com/Jorgvt/korch/tree/master/blob/master/korch/nn.py#L23){target="_blank" style="float:right; font-size:smaller"}

### History



This object inherits from base `dict` to provide a `History` object similar to Keras', 
allowing the automatic logging of the loss and the different metrics during training. 
It's automatically used in `.fit()` (as it is in Keras).

In [2]:
#|output: asis
#| echo: false
show_doc(Module)

---

[source](https://github.com/Jorgvt/korch/tree/master/blob/master/korch/nn.py#L48){target="_blank" style="float:right; font-size:smaller"}

### Module

>      Module (**kwargs)

Modification of PyTorch base `nn.Module` to provide a basic
predefined training loop with logging and a Keras-like interface
to be able to customize the training.
This Module implements as well a `.compile()` method and an `.evaluate()` one. 
All is done to obtain a behaviour as similar to Keras as possible.

In [3]:
#|output: asis
#| echo: false
show_doc(Module.fit)

---

[source](https://github.com/Jorgvt/korch/tree/master/blob/master/korch/nn.py#L105){target="_blank" style="float:right; font-size:smaller"}

### Module.fit

>      Module.fit (trainloader:torch.utils.data.dataloader.DataLoader, epochs,
>                  validationloader=None)

Fit a model to the desired `trainloader` for `epochs` epochs. Returns the corresponding `History`.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| trainloader | DataLoader |  | Training DataLoader. |
| epochs |  |  | Number of epochs to train. |
| validationloader | NoneType | None | Validation DataLoader (optional). |
| **Returns** | **History** |  | **History object with the training dynamics as in Keras.** |

In [4]:
#|output: asis
#| echo: false
show_doc(Module.compile)

---

[source](https://github.com/Jorgvt/korch/tree/master/blob/master/korch/nn.py#L135){target="_blank" style="float:right; font-size:smaller"}

### Module.compile

>      Module.compile (loss:Optional=None,
>                      optimizer:Union[torch.optim.optimizer.Optimizer,NoneType]
>                      =None, metrics=None)

Sets the loss, optimizer and desired metrics to train the model.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| loss | typing.Optional | None | Loss function to be used. |
| optimizer | typing.Union[torch.optim.optimizer.Optimizer, NoneType] | None | Optimizer to be used. |
| metrics | NoneType | None | `MetricCollection` containing the desired metrics. |

In [5]:
#|output: asis
#| echo: false
show_doc(Module.evaluate)

---

[source](https://github.com/Jorgvt/korch/tree/master/blob/master/korch/nn.py#L148){target="_blank" style="float:right; font-size:smaller"}

### Module.evaluate

>      Module.evaluate (dataloader)

Evaluates the model on a set of data.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| dataloader |  | `DataLoader` to evaluate the model with. |
| **Returns** | **typing.Dict** | **Results from the evaluation** |

### Example of usage

We can perform a very simple example using the Fashion MNIST dataset (as is done in the official [PyTorch docs](https://pytorch.org/tutorials/beginner/introyt/trainingyt.html).

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# Create datasets for training & validation, download if necessary
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(training_set, batch_size=128, shuffle=True, num_workers=0)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=256, shuffle=False, num_workers=0)

See that the only different with respect to basic PyTorch is that we're inhereting from our custom [`Module`](https://Jorgvt.github.io/korch/nn.html#module), not from PyTorch's `nn.Module`:

In [ ]:
class SimpleModel(Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Following the usual Keras way, we instantiate the model and compile it, providing the *loss* and the *optimizer*:

In [ ]:
model = SimpleModel()
model.compile(loss=torch.nn.CrossEntropyLoss(),
              optimizer=torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9),
              metrics = MetricCollection([Accuracy()]))

In [ ]:
model

SimpleModel(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (loss_fn): CrossEntropyLoss()
  (metrics): MetricCollection(
    (Accuracy): Accuracy()
  )
)

In [ ]:
model.evaluate(training_loader), model.evaluate(validation_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

({'Accuracy': 0.11770944386275846, 'Loss': 2.3061464941069514},
 {'Accuracy': 0.12353515625, 'Loss': 2.3057154595851896})

In [ ]:
history = model.fit(trainloader=training_loader, epochs=1, validationloader=validation_loader)

Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
history

{'Accuracy': [0.18737784294939752],
 'Loss': [2.2949233914235],
 'Val_Accuracy': [0.2640625],
 'Val_Loss': [2.2760460674762726]}

In [ ]:
model.evaluate(training_loader), model.evaluate(validation_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

({'Accuracy': 0.25996690654932564, 'Loss': 2.276174947905388},
 {'Accuracy': 0.2640625, 'Loss': 2.2760460674762726})